## Hopfield Modell

zunächst werdenen die benötigten Bibliotheken implementiert

In [2]:
import numpy as np
import cv2 #Für Kammera
import tkinter as tk #Für Gui
import matplotlib.pyplot as plt
import tkinter.filedialog
import matplotlib
import threading
import random
matplotlib.use('TkAgg')

Das hier ist lediglich eine funktion um später anschaulicher zu plotten

In [3]:
def on_key(event):
    print(f'Taste {event.key} gedrückt, schließe alle Fenster')
    plt.close('all')

def plot_params(training_patterns, testing_pattern):
    num_training_patterns = len(training_patterns)
    fig1, axs1 = plt.subplots(1, num_training_patterns + 1, figsize=(num_training_patterns * 4, 4))
    
    for i, pattern in enumerate(training_patterns):
        axs1[i].imshow(pattern, cmap='gray')
        axs1[i].set_title(f'Training Pattern {i+1}')
        axs1[i].set_xticks([])
        axs1[i].set_yticks([])
    
    axs1[num_training_patterns].imshow(testing_pattern, cmap='gray')
    axs1[num_training_patterns].set_title('Testing Pattern')
    axs1[num_training_patterns].set_xticks([])
    axs1[num_training_patterns].set_yticks([])

    fig1.suptitle('Netzwerkparamter')
    plt.tight_layout()

def capture_image():
    # Open the webcam (0 is the default device)
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam")
        return None

    # Capture a single frame
    ret, frame = cap.read()
    cap.release()
    
    if not ret:
        print("Error: Could not capture image")
        return None

    # Convert the image to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return gray

def crop_center(image, crop_size):
    """
    Schneidet einen zentralen Ausschnitt aus einem Bild.

    Parameters:
    image (numpy.ndarray): Das Eingabebild als NumPy-Array.
    crop_size (int): Die Größe des quadratischen Ausschnitts.

    Returns:
    numpy.ndarray: Der zentrale Ausschnitt des Bildes.
    """
    h, w = image.shape
    startx = w // 2 - (crop_size // 2)
    starty = h // 2 - (crop_size // 2)
    return image[starty:starty + crop_size, startx:startx + crop_size]



def edit_image(gray_image,matrix_size, crop_size=None):
    if crop_size is None:
        crop_size = 700

    if gray_image is not None:
        img = crop_center(gray_image,crop_size)
        img = np.array(cv2.resize(img, (matrix_size,matrix_size)))
        mean = np.mean(img)
        img = np.where( img >= mean, 1, -1)
        return img
        
    else:
        print("No image captured.")

def flip_values(matrix, probability):
    flipped_matrix = matrix.copy()
    flip_mask = np.random.rand(*matrix.shape) < probability
    flipped_matrix[flip_mask] *= -1
    return flipped_matrix

def flip_matrix_entries(matrix, p):
    # Erstelle eine Kopie der Eingabematrix
    flipped_matrix = np.copy(matrix)
    
    # Get the total number of entries in the matrix
    total_entries = flipped_matrix.size
    
    # Calculate the number of entries to flip
    num_to_flip = int(total_entries * p)
    
    # Get the indices of the matrix entries
    indices = np.indices(flipped_matrix.shape).reshape(2, -1).T
    
    # Randomly choose the indices to flip
    flip_indices = np.random.choice(len(indices), size=num_to_flip, replace=False)
    
    # Flip the selected entries
    for idx in flip_indices:
        i, j = indices[idx]
        if flipped_matrix[i, j] == 1:
            flipped_matrix[i, j] = -1
        elif flipped_matrix[i, j] == -1:
            flipped_matrix[i, j] = 1
            
    return flipped_matrix

def read_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.webp')):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                images.append(img)
            else:
                print(f"Could not read image: {img_path}")
    return images


Hier wird das Programm zur Pixelauswahl Definiert

In [8]:
class PixelGridApp:
    def __init__(self, master, mode):
        self.master = master
        self.master.title("Hopfield Modell Musterauswahl")
        self.grid_size = 5
        self.cell_size = 40 # Jede Zelle ist 40 X 40 Pixel Groß
        self.patterns = []
        self.mode = mode # Modus zur bestimmung ob man im testing oder im trainig modus ist.

        self.grid = np.full((self.grid_size, self.grid_size), 1, dtype=int)
        self.canvases = []

        self.create_widgets()
        self.update_mode_label()
    
    # Im folgenden werden die ganzen Bausteinen und Knöppfe definiert. Den Knöpfen wird jeweils eine Funktion zugeteil die weiter unten definiert sind.
    def create_widgets(self):
        for i in range(self.grid_size):
            row_canvases = []
            for j in range(self.grid_size):
                canvas = tk.Canvas(self.master, width=self.cell_size, height=self.cell_size, bg="white")
                canvas.grid(row=i, column=j)
                canvas.bind("<Button-1>", lambda event, i=i, j=j: self.toggle_pixel(i, j))
                canvas.create_text(self.cell_size//2, self.cell_size//2, fill="black", font=("Helvetica", 16))
                row_canvases.append(canvas)
            self.canvases.append(row_canvases)

        self.save_button = tk.Button(self.master, text="Save Pattern", command=self.save_pattern)
        self.save_button.grid(row=self.grid_size, column=0, columnspan=self.grid_size // 2)

        self.reset_button = tk.Button(self.master, text="Reset Grid", command=self.reset_grid)
        self.reset_button.grid(row=self.grid_size, column=self.grid_size // 2, columnspan=self.grid_size // 2)

        self.save_file_button = tk.Button(self.master, text="Save to File", command=self.prompt_save_file)
        self.save_file_button.grid(row=self.grid_size + 1, column=0, columnspan=self.grid_size // 2)

        self.load_file_button = tk.Button(self.master, text="Load from File", command=self.prompt_load_file)
        self.load_file_button.grid(row=self.grid_size + 1, column=self.grid_size // 2, columnspan=self.grid_size // 2)

        self.finish_button = tk.Button(self.master, text="Finish", command=self.finish)
        self.finish_button.grid(row=self.grid_size + 2, column=0, columnspan=self.grid_size)

        self.mode_label = tk.Label(self.master, text="")
        self.mode_label.grid(row=self.grid_size + 3, column=0, columnspan=self.grid_size)
    
    def update_mode_label(self):
        if self.mode == "training":
            self.mode_label.config(text="Mode: Training")
        elif self.mode == "testing":
            self.mode_label.config(text="Mode: Testing")

    def toggle_pixel(self, i, j):
        if self.grid[i, j] == 1:
            self.grid[i, j] = -1
            self.canvases[i][j].config(bg="black")
            self.canvases[i][j].itemconfig(1, fill="white")
        else:
            self.grid[i, j] = 1
            self.canvases[i][j].config(bg="white")
            self.canvases[i][j].itemconfig(1, fill="black")
    
    def save_pattern(self):
        self.patterns.append(self.grid.copy())
        print("Pattern saved:")
        print(self.grid)
        if self.mode == 'testing':
            self.testing_mode()
    
    def reset_grid(self):
        self.grid.fill(1)
        for row_canvases in self.canvases:
            for canvas in row_canvases:
                canvas.config(bg="white")
                canvas.itemconfig(1, fill="black")
        print("Grid reset")
    
    def prompt_save_file(self):
        filename = tk.filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text files", "*.txt"), ("All files", "*.*")])
        if filename:
            self.save_patterns_to_file(filename)
            if self.mode == "training":
                self.switch_to_testing_mode()
            if self.mode == "testing":
                self.testing_mode()
    
    def save_patterns_to_file(self, filename):
        np.savetxt(filename, np.array(self.patterns).reshape(len(self.patterns), -1), fmt='%d')
        print(f"Patterns saved to {filename}")
    
    def prompt_load_file(self):
        filename = tk.filedialog.askopenfilename(filetypes=[("Text files", "*.txt"), ("All files", "*.*")])
        if filename:
            self.load_patterns_from_file(filename)
            if self.mode == "training":
                self.switch_to_testing_mode()
            if self.mode == "testing":
                self.testing_mode()
            
    
    def load_patterns_from_file(self, filename):
        data = np.loadtxt(filename, dtype=int)
        if self.mode == "testing":
            self.patterns = data.reshape(1, self.grid_size, self.grid_size).tolist()
        else:
            num_patterns = data.shape[0]
            self.patterns = data.reshape(num_patterns, self.grid_size, self.grid_size).tolist()
        print(f"Patterns loaded from {filename}")
    
    def switch_to_testing_mode(self):
        self.training_patterns = np.array(self.patterns)
        self.mode = "testing"
        self.patterns = []
        self.update_mode_label()
        self.reset_grid()
        print("Switched to Testing mode")

    def testing_mode(self):
        self.testing_pattern = np.array(self.patterns)
        print('Testing Pattern saved in Array')

    
    def finish(self):
        try:
            self.testing_pattern
        except:
            self.testing_pattern = np.zeros((5, 5))
            print('Error: Nichtsts gespeichert')
        try:
            self.training_patterns
        except:
            self.training_patterns = np.zeros((5, 5))
            print('Error: Nichtsts gespeichert')   
        self.master.quit()  # Beendet die Tkinter-Mainloop
        self.master.destroy()  # Zerstört das Tkinter-Fenster


Jetzt wird zunächst das Hopfieldmodel implementiert

In [119]:
class HopfieldNetwork:
    
    #Initierungsfunktion
    def __init__(self, num_neurons, steps):
        self.num_neurons = num_neurons
        self.matrix_size = int(np.sqrt(self.num_neurons))
        self.weights = np.zeros((num_neurons, num_neurons))
        self.steps = steps
        self.plot_nth_time = 1  # Standardwert für plot_nth_time
        self.patterns = []

    #Trainig durch die Hebbsche Regel. Normiert auf die Anzahl der Neuronen. Input sind gegebene Patterns
    def train(self, patterns):
        self.patterns = patterns
        for p in patterns:
            p = p.flatten()  # Ensuring pattern is a column vector
            if len(p) != self.num_neurons:
                raise ValueError(f"Pattern size {len(p)} does not match the number of neurons {self.num_neurons}.")
            self.weights += np.outer(p, p)
        self.weights /= self.num_neurons
        np.fill_diagonal(self.weights, 0)  # No self-connections
        

    def show_as_matrix(self,pattern):
        matrix = np.reshape(pattern, (self.matrix_size, self.matrix_size))
        return matrix
    
    def predict_beta(self, pattern, activation_function, init_beta, trainig_steps):
        s = pattern.flatten().copy()
        m_0_max = self.m_biggest(self.patterns, s)
        m_arr_max = [m_0_max]
        beta =init_beta
        l = len(range(self.steps))
        beta_step = init_beta/l
        beta =init_beta
        betas = [init_beta]
        max_m_t =[]
        for i in range(trainig_steps):
            for i in range(self.steps):
                a = np.dot(self.weights, s)
                s = activation_function(a, beta)
                m_max = self.m_biggest(self.patterns, s)
                m_arr_max.append(m_max)
                s = s.flatten()
            betas.append(beta)
            beta = beta-beta_step
            m_end = m_arr_max[self.steps-1]
            max_m_t.append(m_end)
        m_max_index = np.argmax(np.abs(max_m_t))
        # Element mit dem größten absoluten Wert auswählen
        m_max_end = max_m_t[m_max_index]
        b_crit = betas[m_max_index]
        return  b_crit, m_max_end  



    def predict(self, pattern, activation_function, axs=None, beta=None, plot_nth_time=None):
        if plot_nth_time is None:
            plot_nth_time = 1
        s = pattern.flatten().copy()  # Kopie des Musters und Umformung zu Spaltenvektor
        eng_array = np.array([self.energy(s)])
        m_0 = self.m_mean(self.patterns, s)
        m_0_max = self.m_biggest(self.patterns, s)
        m_array = [m_0]
        m_arr_max = [m_0_max]
        plot_index = 0
        for i in range(self.steps):
            a = np.dot(self.weights, s)
            s = activation_function(a, beta)
            s = s.flatten()
            eng = self.energy(s)
            m = self.m_mean(self.patterns, s)
            m_max = self.m_biggest(self.patterns, s)
            m_array.append(m)
            m_arr_max.append(m_max)
            eng_array = np.append(eng_array, eng)
            
            if axs is not None and i % plot_nth_time == 0:
                plot_index = int(i/plot_nth_time)
                axs[plot_index].imshow(self.show_as_matrix(s), cmap='gray')
                axs[plot_index].set_title(f'Nach {i+1} Steps')
                axs[plot_index].set_xticks([])
                axs[plot_index].set_yticks([])
                plt.tight_layout()
                
        self.eng_array = eng_array
        self.m_array = m_array
        self.m_array_max = m_arr_max
        return s.flatten()  # Rückgabe als flaches Array

    
    def m_singel(self, pattern, state):
        pattern = pattern.flatten().copy()
        state = state.flatten().copy()
        return np.dot(pattern, state) /self.num_neurons
    
    
    def m_mean(self, patterns, state):
        m = [self.m_singel(pattern, state) for pattern in patterns]
        m_mean = np.mean(m)
        return m_mean
    
    def m_biggest(self, patterns, state):
        m = [self.m_singel(pattern, state) for pattern in patterns]
        # Index des Elements mit dem größten absoluten Wert finden
        m_max_index = np.argmax(np.abs(m))
        # Element mit dem größten absoluten Wert auswählen
        m_max = m[m_max_index]
        return m_max

    def energy(self, pattern):
       
        return -0.5 * np.dot(np.dot(pattern.T, self.weights), pattern)
    

#Aktivierungsfunktionen
def tanh(a):
    return np.tanh(a)
    
def heviside(a, g=None):
    return np.where(a >= 0, 1, -1)

def stochastic(a, beta):
    def prop(a, beta):
        return 1/(1+np.exp(-2*beta*a))
    return 1 if random.random() < prop(a, beta) else -1

def stochastic2(a, beta):
    def prop(a, beta):
        return 1/(1+np.exp(-2*beta*a))
    
    # Ensure the function returns an array of the same shape as `a`
    prob_array = prop(a, beta)
    
    # Generate random values for comparison
    random_values = np.random.rand(*a.shape)
    
    # Perform element-wise comparison
    return np.where(random_values < prob_array, 1, -1)
    




In [94]:
#images
#Hopfield Network starten mit gewünschten Parameter
network = HopfieldNetwork(num_neurons= 10000, steps= 4)
steps = network.steps 
matrix_size = network.matrix_size  
print(matrix_size)     
#Bildaxe und Plot initalisieren
fig1, axs1 = plt.subplots(1, 2, figsize=(3 * 4, 4))
fig2, axs2 = plt.subplots(1, steps, figsize=(3 * 4, 4))

# Bild Aufnehmen und lernen
gray_image = capture_image()
learn_image = edit_image(gray_image, matrix_size)
print(matrix_size)
axs1[0].imshow(learn_image, cmap='gray')
axs1[0].set_title('Aufgenommenes Bild')
network.train([learn_image])
print(learn_image)

#Rauschen drautftun

noise_level = 0.9
noise_img = flip_values(learn_image,noise_level)

axs1[1].imshow(noise_img, cmap='gray')
axs1[1].set_title(f'Bild mit Einem Noiselevel von {noise_level*100}% ')

fig1.suptitle('Netzwerkparamter')

#Netzwerk Anwenden

result = network.predict(noise_img,heviside, axs=axs2)
fig2.suptitle('Netzwerkausgabe')
plt.show()
#print(network.weights)



100
100
[[ 1  1  1 ...  1  1  1]
 [ 1  1  1 ...  1  1  1]
 [ 1  1  1 ...  1  1  1]
 ...
 [-1 -1 -1 ... -1 -1 -1]
 [-1 -1 -1 ... -1 -1 -1]
 [-1 -1 -1 ... -1 -1 -1]]


In [107]:
plt.close('all')
root = tk.Tk()
app = PixelGridApp(root, "training")
root.mainloop()


training = app.training_patterns
testing = app.testing_pattern[0]


network = HopfieldNetwork(num_neurons=25, steps=7)
steps = network.steps

network.train(training)
plot_params(training_patterns=training, testing_pattern=testing)
fig2, axs2 = plt.subplots(1, steps, figsize=(len(training) * 4, 4))
result = network.predict(testing,heviside, axs=axs2)
fig2.suptitle('Netzwerkausgabe')
fig3, axs3 = plt.subplots(1,1)
axs4 = axs3.twinx()
axs4.plot(np.arange(steps+1), network.m_array_max, color ='forestgreen', label = r'm$_{max}$')
axs4.plot(np.arange(steps+1), network.m_array, color ='lightgreen', label = r'm$_{mean}$')
axs4.legend()
axs3.grid()
fig3.suptitle('Energiefunktion')
axs3.set_xlabel('Schritte')
axs3.set_ylabel('Energie')
axs3.plot(np.arange(steps+1),network.eng_array)

plt.show()

print(network.m_array)


Patterns loaded from /Users/phileasmeier/Documents/UNI/Neurale_Netzwerke/Hopfield_Model/Hopefield_Model/McKay_training.txt
Grid reset
Switched to Testing mode
Testing Pattern saved in Array
Pattern saved:
[[ 1  1  1  1  1]
 [ 1  1  1  1  1]
 [-1  1 -1  1  1]
 [ 1 -1  1  1  1]
 [ 1  1  1  1  1]]
Testing Pattern saved in Array
[-0.12, -0.2, 0.04, 0.04]
4
[-0.44, -0.52, -0.28, 0.52]
4
[-0.28, -0.68, -0.44, 0.52]
4
[-0.12, -0.68, -0.6, 0.52]
4
[-0.12, -0.68, -0.6, 0.52]
4
[-0.12, -0.68, -0.6, 0.52]
4
[-0.12, -0.68, -0.6, 0.52]
4
[-0.12, -0.68, -0.6, 0.52]
4
[-0.060000000000000005, -0.18, -0.22000000000000003, -0.21999999999999997, -0.21999999999999997, -0.21999999999999997, -0.21999999999999997, -0.21999999999999997]


Stochastic

In [127]:
plt.close('all')
import os
import cv2
import numpy as np

network = HopfieldNetwork(num_neurons= 2500, steps= 30)
steps = network.steps 
beta_init = 9
plot_times = 7
t = 0.1
testing_steps = 30


    
# Beispielverwendung
folder_path = 'Bilder'
images = read_images_from_folder(folder_path)
gray_images = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in images]
gray_images = [edit_image(img,network.matrix_size, 700) for img in gray_images]

'''fig, ax = plt.subplots(1, len(gray_images),figsize=(len(gray_images) * 4, 4))
for i,img in enumerate(gray_images):
    ax[i].imshow(img)'''
training = gray_images   

network.train(training)
beta_crit, m_max = network.predict_beta(testing, stochastic2, init_beta= beta_init, trainig_steps= testing_steps)
print(beta_crit, m_max)
#testing = flip_values(training[3],t)
testing = flip_matrix_entries(training[8],t)
plot_params(training,testing)
fig2, axs2 = plt.subplots(1, int(np.floor((steps/plot_times)))+1,  figsize=(int(np.floor((steps/plot_times)))*4, 4))
result = network.predict(testing,stochastic2, axs=axs2, beta=10 , plot_nth_time=plot_times)
# Überprüfen Sie die Anzahl der geladenen Bilder
print(f"Number of images read: {len(images)}")

fig3, axs3 = plt.subplots(1,1)
axs3.grid()
fig3.suptitle('Energiefunktion')
axs3.set_xlabel('Schritte')
axs3.set_ylabel('Energie')
#eng_x = np.delete(np.arange(steps+1),0)
eng_x = np.arange(steps+1)
#eng_y = np.delete(network.eng_array,0)
eng_y = network.eng_array
axs3.plot(eng_x,eng_y)

axs4 = axs3.twinx()
axs4.plot(np.arange(steps+1), network.m_array_max, color ='forestgreen', label = r'm$_{max}$')
axs4.plot(np.arange(steps+1), network.m_array, color ='lightgreen', label = r'm$_{mean}$')
axs4.set_ylabel('order parameter')
axs4.legend()
fig3.canvas.mpl_connect('key_press_event', on_key)
fig2.canvas.mpl_connect('key_press_event', on_key)
#fig1.canvas.mpl_connect('key_press_event', on_key)
plt.show()
print(network.m_array)



9 0.688
Number of images read: 9


KeyboardInterrupt: 